# Natures Eye

## Introduction

### Import Libraries and Define Functions

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import seaborn as sns
import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models, layers, regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread 
from PIL import Image

# Neural network libraries

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, losses, metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, classification_report, recall_score, ConfusionMatrixDisplay
from sklearn.dummy import DummyClassifier
np.random.seed(42)

In [30]:
from keras.layers.convolutional import Conv2D

In [33]:
from keras.layers import Activation, Dense

In [31]:
def show_metrics(history):
    results = history.history
    plt.figure()
    plt.plot(results['val_accuracy'], label = 'Validation Accuracy')
    plt.plot(results['accuracy'], label = 'Train Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show();
    
    plt.figure()
    plt.plot(results['val_recall'], label = 'Validation Recall')
    plt.plot(results['recall'], label = 'Train Recall')
    plt.title('Recall Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Recall')
    plt.legend()
    plt.show();
    
    plt.figure()
    plt.plot(results['val_loss'], label = 'Validation Loss')
    plt.plot(results['loss'], label = 'Train Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show();

In [5]:
# Create function to display confusion matrices for models
def conf_mat(model):
    y_pred = model.predict(test_set)
    y_pred = (y_pred>0.5).astype(np.int)
    display(ConfusionMatrixDisplay(confusion_matrix(test_set.classes, y_pred), display_labels=['Normal', 'Pneumonia']).plot())

## Data Understanding

## Data Cleaning

### Data Loading

In [6]:
#train_data_dir = 'ZhangLabData/CellData/chest_xray/train'
train_path = 'archive/animals'

#Get all the data in the directory data/validation (624 images), and reshape them
#test_generator = ImageDataGenerator().flow_from_directory(
#       test_data_dir, 
#       target_size=(64, 64), batch_size=624, color_mode='grayscale', class_mode='binary')

#Get all the data in the directory data/train (5232 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
       train_path, 
       target_size=(64, 64), color_mode='rgb', class_mode='categorical')

#Create the datasets
train_images, train_labels = next(train_generator)
#test_images, test_labels = next(test_generator)

Found 5400 images belonging to 90 classes.


In [7]:
#train_labels[:10]
train_labels[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)

In [8]:
train_generator.class_indices

{'antelope': 0,
 'badger': 1,
 'bat': 2,
 'bear': 3,
 'bee': 4,
 'beetle': 5,
 'bison': 6,
 'boar': 7,
 'butterfly': 8,
 'cat': 9,
 'caterpillar': 10,
 'chimpanzee': 11,
 'cockroach': 12,
 'cow': 13,
 'coyote': 14,
 'crab': 15,
 'crow': 16,
 'deer': 17,
 'dog': 18,
 'dolphin': 19,
 'donkey': 20,
 'dragonfly': 21,
 'duck': 22,
 'eagle': 23,
 'elephant': 24,
 'flamingo': 25,
 'fly': 26,
 'fox': 27,
 'goat': 28,
 'goldfish': 29,
 'goose': 30,
 'gorilla': 31,
 'grasshopper': 32,
 'hamster': 33,
 'hare': 34,
 'hedgehog': 35,
 'hippopotamus': 36,
 'hornbill': 37,
 'horse': 38,
 'hummingbird': 39,
 'hyena': 40,
 'jellyfish': 41,
 'kangaroo': 42,
 'koala': 43,
 'ladybugs': 44,
 'leopard': 45,
 'lion': 46,
 'lizard': 47,
 'lobster': 48,
 'mosquito': 49,
 'moth': 50,
 'mouse': 51,
 'octopus': 52,
 'okapi': 53,
 'orangutan': 54,
 'otter': 55,
 'owl': 56,
 'ox': 57,
 'oyster': 58,
 'panda': 59,
 'parrot': 60,
 'pelecaniformes': 61,
 'penguin': 62,
 'pig': 63,
 'pigeon': 64,
 'porcupine': 65,
 'pos

## Create validation set from training set

In [9]:
# Create validation set from training set

X = train_images
y = train_labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## dummy classifer/regressor for baseline

In [10]:
#dummy classifer/regressor for baseline
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='uniform')
dummy.fit(X_train, y_train)

DummyClassifier(strategy='uniform')

In [11]:
dummy.score(X_val, y_val)

0.0

In [12]:
dummy.predict(X_val)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 1.

In [14]:
plot_confusion_matrix(dummy, X_val, y_val)

ValueError: multilabel-indicator is not supported

##  Creation of neural network model

In [15]:
# Creation of neural network model
simple_model = models.Sequential([
    layers.Flatten(input_shape=(64, 64, 1)),
    layers.Dense(20, activation='relu'),
    layers.Dense(7, activation='relu'),
    layers.Dense(5, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

simple_model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['categorical_accuracy', metrics.Recall(name='recall'), metrics.AUC(name='auc'), metrics.AUC(name='prc', curve='PR')])

simple_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 20)                81940     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 147       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 40        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 82,133
Trainable params: 82,133
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Fit the train_set data to the simple model built above and use validation set 
simple_history = simple_model.fit(X_train,
                    epochs=3,
                    batch_size=32,
                    validation_data=y_train.all())

Epoch 1/3


ValueError: in user code:

    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\danie\anaconda3\envs

In [ ]:
show_metrics(simple_history)

## Softmax

In [34]:
model = keras.models.Sequential()

model.add(Conv2D(32, kernel_size=5, strides=2, activation='relu', input_shape=(268, 182, 3)))
model.add(Conv2D(64, kernel_size=3, strides=1, activation='relu'))       

model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))   # Final Layer using Softmax

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  

In [35]:
# Fit the train_set data to the simple model built above and use validation set 
simple_history = simple_model.fit(X_train,
                    y_train,
                    epochs=3,
                    batch_size=32,
                    validation_data= (X_val, y_val))

Epoch 1/3


ValueError: in user code:

    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\danie\anaconda3\envs

## New Model

In [1]:
conda install pytorch torchvision cpuonly -c pytorch

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\danie\anaconda3\envs\learn-env

  added / updated specs:
    - cpuonly
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.26 |       haa95532_2         115 KB
    certifi-2021.10.8          |   py38haa95532_2         152 KB
    cpuonly-2.0                |                0           2 KB  pytorch
    libuv-1.40.0               |       he774522_0         255 KB
    openssl-1.1.1m             |       h2bbff1b_0         4.8 MB
    pytorch-1.10.1             |      py3.8_cpu_0       193.1 MB  pytorch
    pytorch-mutex-1.0          |              cpu           3 KB  pytorch
    torchvision-0.11.2         |         py38_cpu         7.2 MB  pytorch
    typing_extensions-3.10.0.2 |     pyh06a4308_0          31 KB
    --------------------------------



==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda





pytorch-1.10.1       | 193.1 MB  | 6          |   7% 
pytorch-1.10.1       | 193.1 MB  | 6          |   7% 
pytorch-1.10.1       | 193.1 MB  | 6          |   7% 
pytorch-1.10.1       | 193.1 MB  | 7          |   7% 
pytorch-1.10.1       | 193.1 MB  | 7          |   7% 
pytorch-1.10.1       | 193.1 MB  | 7          |   7% 
pytorch-1.10.1       | 193.1 MB  | 7          |   7% 
pytorch-1.10.1       | 193.1 MB  | 7          |   7% 
pytorch-1.10.1       | 193.1 MB  | 7          |   8% 
pytorch-1.10.1       | 193.1 MB  | 7          |   8% 
pytorch-1.10.1       | 193.1 MB  | 7          |   8% 
pytorch-1.10.1       | 193.1 MB  | 7          |   8% 
pytorch-1.10.1       | 193.1 MB  | 8          |   8% 
pytorch-1.10.1       | 193.1 MB  | 8          |   8% 
pytorch-1.10.1       | 193.1 MB  | 8          |   8% 
pytorch-1.10.1       | 193.1 MB  | 8          |   8% 
pytorch-1.10.1       | 193.1 MB  | 8          |   8% 
pytorch-1.10.1       | 193.1 MB  | 8          |   9% 
pytorch-1.10.1       | 193.

pytorch-1.10.1       | 193.1 MB  | ####4      |  45% 
pytorch-1.10.1       | 193.1 MB  | ####4      |  45% 
pytorch-1.10.1       | 193.1 MB  | ####5      |  45% 
pytorch-1.10.1       | 193.1 MB  | ####5      |  45% 
pytorch-1.10.1       | 193.1 MB  | ####5      |  46% 
pytorch-1.10.1       | 193.1 MB  | ####6      |  46% 
pytorch-1.10.1       | 193.1 MB  | ####6      |  46% 
pytorch-1.10.1       | 193.1 MB  | ####6      |  47% 
pytorch-1.10.1       | 193.1 MB  | ####6      |  47% 
pytorch-1.10.1       | 193.1 MB  | ####7      |  47% 
pytorch-1.10.1       | 193.1 MB  | ####7      |  48% 
pytorch-1.10.1       | 193.1 MB  | ####7      |  48% 
pytorch-1.10.1       | 193.1 MB  | ####8      |  48% 
pytorch-1.10.1       | 193.1 MB  | ####8      |  48% 
pytorch-1.10.1       | 193.1 MB  | ####8      |  49% 
pytorch-1.10.1       | 193.1 MB  | ####8      |  49% 
pytorch-1.10.1       | 193.1 MB  | ####9      |  49% 
pytorch-1.10.1       | 193.1 MB  | ####9      |  50% 
pytorch-1.10.1       | 193.1

In [3]:
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\danie\anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.11.0               |   py38haa95532_0        14.4 MB
    ------------------------------------------------------------
                                           Total:        14.4 MB

The following packages will be UPDATED:

  conda                               4.10.3-py38haa95532_0 --> 4.11.0-py38haa95532_0




conda-4.11.0         | 14.4 MB   |            |   0% 
conda-4.11.0         | 14.4 MB   |            |   0% 
conda-4.11.0         | 14.4 MB   | 3          |   3% 
conda-4.11.0         | 14.4 MB   | 6          |   6% 
conda-4.11.0         | 14.4 MB   | 9          |  10% 
conda-4.11.0         | 14.4 MB   | #2         |  13% 
conda-4.11.0         | 14.4 MB   | #5         |  16% 
c

In [37]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'torch'

In [ ]:
data_dir = ''archive/animals'
classes = os.listdir(data_dir)

In [ ]:
train_transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(100),             # resize shortest side to 100 pixels
        transforms.CenterCrop(100),         # crop longest side to 100 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

In [ ]:
dataset = ImageFolder(data_dir, transform=train_transform)
print('Size of training dataset :', len(dataset))

In [ ]:
# view one image shape of the dataset.
img, label = dataset[100]
print(img.shape)

In [ ]:
# function for the showing the image.
def show_image(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
show_image(*dataset[200])

In [ ]:
torch.manual_seed(20)
val_size = len(dataset)//10
test_size = len(dataset)//5
train_size = len(dataset) - val_size -test_size

In [ ]:
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])
len(train_ds), len(val_ds),len(test_ds)  

In [ ]:
batch_size = 64
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
for images, labels in train_loader:
    fig, ax = plt.subplots(figsize=(18,10))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
    break

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
torch.cuda.is_available()

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
input_size = 3*100*100
output_size = 91 # Number of classe

## Linear Model

In [ ]:
class Model(ImageClassificationBase):
    def __init__(self, input_size, output_size):
        super().__init__()
        # hidden layer
        self.in_layer = nn.Linear(input_size, 8384)
        self.hidden1 = nn.Linear(8384, 4192)
        self.hidden2 = nn.Linear(4192, 2096)
        self.hidden3 = nn.Linear(2096, 1048)
        self.out_layer = nn.Linear(1048, output_size)
        
    def forward(self, xb):
        # Flatten images into vectors
        out = xb.view(xb.size(0), -1)
        out = self.in_layer(out)
        out = self.hidden1(F.relu(out))
        out = self.hidden2(F.relu(out))
        out = self.hidden3(F.relu(out))
        out = self.out_layer(F.relu(out))
        return out

In [ ]:
model = to_device(Model(input_size, output_size), device)

In [ ]:
model

In [ ]:
history = [evaluate(model, val_loader)]
history

In [ ]:
history += fit(7, 0.01, model, train_loader, val_loader)

In [ ]:
history += fit(8, 0.001, model, train_loader, val_loader)

In [ ]:
history += fit(3, 0.0001, model, train_loader, val_loader)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')
    plt.show()
    
    
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
    plt.show()

In [ ]:
plot_accuracies(history)

In [ ]:
plot_losses(history)

In [ ]:
evaluate(model, test_loader)

In [ ]:
train_generator.class_indices

In [ ]:
plt.figure(figsize=(16,16))
for n in range(5):
    normal_img = os.listdir(train_data_dir/bat')[n]
    normal_img_address = train_data_dir
    normal_load = Image.open(normal_img_address, 'r')
    ax = plt.subplot(5,5,n+1)
    plt.imshow(normal_load, cmap ='rgb')
    plt.title("NORMAL")
    plt.axis("off")

In [ ]:
normal_load = Image.open(train_bat, 'r')
#ax = plt.subplot(5,5,n+1)
#plt.imshow(normal_load, cmap ='rgb')
#plt.title("NORMAL")
#plt.axis("off")
type(normal_load)

## Modeling

In [ ]:
# sklearn dummy classifer that will always return the most frequent class (pneumonia)
baseline_model = DummyClassifier(strategy='most_frequent')
baseline_model.fit(train_images, train_labels)

In [ ]:
# Scoring dummy model for accuracy and recall
base_acc = baseline_model.score(test_images, test_labels)
print(base_acc)

# Generate predicts using model
y_pred_base = baseline_model.predict(test_images)

# Recall score
base_recall = recall_score(test_labels, y_pred_base)
print(base_recall)

In [ ]:
# Visualize confusion matrix for dummy model
plot_confusion_matrix(baseline_model, test_images, test_labels)

### Iterations

## Evaluation

## Summary

In [ ]:
train_path = 'archive/animals'

In [ ]:
# Utlize ImageDataGenerator from keras.preprocessing
img_dgen = ImageDataGenerator(rescale=1./255, validation_split = 0.20)

# Creation of train set, note the subset argument due to the validation split
train_set = img_dgen.flow_from_directory(train_path, target_size=(64, 64), 
                                         color_mode='grayscale', subset='training')

# Creation of validation set from 20% of the train data
validation_set = img_dgen.flow_from_directory(train_path, target_size=(64, 64), 
                                         color_mode='grayscale', subset='validation')

# Creation of test set from the test path, shuffle set to false for use in future model predictions
#test_set = img_dgen.flow_from_directory(test_path, target_size=(64, 64), 
#                                         color_mode='grayscale', class_mode='binary', shuffle=False)

In [ ]:
# Batch size set to the number of images found above in order to not batch data for the dummy model. Validation set not needed for dummy model
train_set_dummy = img_dgen.flow_from_directory(train_path,
                                               target_size=(64, 64),
                                               color_mode='rgb', 
                                               class_mode='categorical', 
                                               subset='training', 
                                               batch_size=4000)

test_set_dummy = img_dgen.flow_from_directory(validation_set, 
                                              target_size=(64, 64),
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              shuffle=False,
                                              batch_size=1400)

# Split train and test dummy sets into images and corresponding labels for dummy model training
train_images, train_labels = next(train_set_dummy)
test_images, test_labels = next(test_set_dummy)

In [ ]:
# Scoring dummy model for accuracy and recall
base_acc = baseline_model.score(test_images, test_labels)
print(base_acc)

# Generate predicts using model
y_pred_base = baseline_model.predict(test_images)

# Recall score
base_recall = recall_score(test_labels, y_pred_base)
print(base_recall)